# Setup and Installations

In [ ]:
!pip install torch torchvision torchaudio pytorch-lightning timm numpy pandas matplotlib scikit-learn

import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
from google.colab import drive
import pytorch_lightning as pl
from timm import create_model
from torchmetrics.classification import BinaryAccuracy, BinaryAUROC
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from timm import create_model

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

print(f"PyTorch Version: {torch.__version__}")
print(f"PyTorch Lightning Version: {pl.__version__}")

PyTorch Version: 2.9.0+cu126
PyTorch Lightning Version: 2.5.6


In [ ]:
drive.mount('/content/drive')
print("✅ Google Drive mounted successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully!


In [ ]:
DATA_ROOT = "/content/drive/MyDrive/dataset/data/"

print(f"Looking for data at: {DATA_ROOT}")
if os.path.exists(DATA_ROOT):
    print(f" Found data directory!")
    print(f"Contents: {os.listdir(DATA_ROOT)}")
else:
    print(f"Directory not found!")

SPLITS = {
    'train': os.path.join(DATA_ROOT, 'train'),
    'valid': os.path.join(DATA_ROOT, 'valid'),
    'test': os.path.join(DATA_ROOT, 'test')
}

CLASS_NAMES = {'real': 0, 'fake': 1}

split_data = {}
for split_name, split_dir in SPLITS.items():
    data = []
    print(f"\n Processing {split_name} split...")

    for class_name, label in CLASS_NAMES.items():
        class_dir = os.path.join(split_dir, class_name)
        if not os.path.isdir(class_dir):
            print(f"Directory not found: {class_dir}")
            continue

        # Load image files
        files = [f for f in os.listdir(class_dir) if f.endswith(('.jpg', '.png', '.jpeg', '.JPG', '.PNG', '.JPEG'))]
        print(f"{class_name}: {len(files)} images")

        for filename in files:
            data.append({
                'path': os.path.join(class_dir, filename),
                'label': label,
                'label_name': class_name
            })

    split_data[split_name] = pd.DataFrame(data)

train_df = split_data['train']
val_df = split_data['valid']
test_df = split_data['test']

print(f"\n{'='*60}")
print(f"SUCCESS! Data loaded from Google Drive")
print(f"{'='*60}")
print(f" Train: {len(train_df):,} images")
print(f" Valid: {len(val_df):,} images")
print(f" Test: {len(test_df):,} images")
print(f" Total: {len(train_df) + len(val_df) + len(test_df):,} images")
print(f"{'='*60}")

Looking for data at: /content/drive/MyDrive/dataset/data/
 Found data directory!
Contents: ['valid', 'train', 'test']

 Processing train split...
real: 3500 images
fake: 3500 images

 Processing valid split...
real: 750 images
fake: 750 images

 Processing test split...
real: 750 images
fake: 750 images

SUCCESS! Data loaded from Google Drive
 Train: 7,000 images
 Valid: 1,500 images
 Test: 1,500 images
 Total: 10,000 images


In [ ]:
print("\n--- Dataset Summary ---")
print(f"Total Images: {len(train_df) + len(val_df) + len(test_df)}")
print(f"Train set size: {len(train_df)}")
print(train_df['label_name'].value_counts())
print(f"\nValidation set size: {len(val_df)}")
print(val_df['label_name'].value_counts())
print(f"\nTest set size: {len(test_df)}")
print(test_df['label_name'].value_counts())


--- Dataset Summary ---
Total Images: 10000
Train set size: 7000
label_name
real    3500
fake    3500
Name: count, dtype: int64

Validation set size: 1500
label_name
real    750
fake    750
Name: count, dtype: int64

Test set size: 1500
label_name
real    750
fake    750
Name: count, dtype: int64


In [ ]:
train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)

In [ ]:
IMG_SIZE = 224
NORM_MEAN = [0.485, 0.456, 0.406]
NORM_STD = [0.229, 0.224, 0.225]

class FaceForgeryDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['path']
        label = self.df.iloc[idx]['label']

        try:
            image = Image.open(img_path).convert('RGB')
        except:
            print(f"Warning: Could not read image at {img_path}. Skipping.")
            return self.__getitem__(random.randint(0, len(self) - 1))

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

class FaceForgeryDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=32):
        super().__init__()
        self.batch_size = batch_size

        self.train_transform = transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
            transforms.ToTensor(),
            transforms.Normalize(mean=NORM_MEAN, std=NORM_STD)
        ])

        self.val_test_transform = transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize(mean=NORM_MEAN, std=NORM_STD)
        ])

    def setup(self, stage=None):
        self.train_dataset = FaceForgeryDataset("train.csv", transform=self.train_transform)
        self.val_dataset = FaceForgeryDataset("val.csv", transform=self.val_test_transform)
        self.test_dataset = FaceForgeryDataset("test.csv", transform=self.val_test_transform)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=2, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=2, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=2, pin_memory=True)

BATCH_SIZE = 32
data_module = FaceForgeryDataModule(batch_size=BATCH_SIZE)
data_module.setup()

In [ ]:
NUM_CLASSES = 2

class ClassifierMixin(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = BinaryAccuracy()
        self.auroc = BinaryAUROC()

    def common_step(self, batch, batch_idx):
        images, labels = batch
        logits = self(images)
        loss = self.loss_fn(logits, labels)
        preds = torch.argmax(logits, dim=1)

        acc = self.accuracy(preds, labels)
        auroc_score = self.auroc(logits[:, 1], labels)

        return loss, acc, auroc_score

    def training_step(self, batch, batch_idx):
        loss, acc, _ = self.common_step(batch, batch_idx)
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('train_acc', acc, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc, auroc_score = self.common_step(batch, batch_idx)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val_acc', acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val_auroc', auroc_score, on_step=False, on_epoch=True, prog_bar=True)

    def test_step(self, batch, batch_idx):
        loss, acc, auroc_score = self.common_step(batch, batch_idx)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        self.log('test_acc', acc, on_step=False, on_epoch=True)
        self.log('test_auroc', auroc_score, on_step=False, on_epoch=True)

    def configure_optimizers(self):
        for param in self.backbone.parameters():
            param.requires_grad = False

        optimizer = torch.optim.AdamW(self.classifier.parameters(), lr=self.learning_rate)
        return optimizer


class ResNetClassifier(ClassifierMixin):
    def __init__(self, model_name='resnet50', num_classes=NUM_CLASSES, learning_rate=1e-4):
        ClassifierMixin.__init__(self)
        self.learning_rate = learning_rate

        self.backbone = create_model(model_name, pretrained=True, num_classes=0)

        num_features = self.backbone.num_features

        self.classifier = nn.Linear(num_features, num_classes)

    def forward(self, x):
        features = self.backbone(x)
        logits = self.classifier(features)
        return logits

model = ResNetClassifier(model_name='resnet50', learning_rate=1e-4)

print(f"Selected Model: {type(model).__name__}")
print(f"Backbone Type: {type(model.backbone).__name__}")
print(f"Classifier Head: {model.classifier}")

Selected Model: ResNetClassifier
Backbone Type: ResNet
Classifier Head: Linear(in_features=2048, out_features=2, bias=True)


In [ ]:
print("=" * 60)
print("PHASE 1: Training Classification Head Only")
print("=" * 60)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints/',
    filename='phase1-best-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min',
)

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min'
)

gpus = 1 if torch.cuda.is_available() else 0
print(f"Using {gpus} GPU(s).")

trainer = pl.Trainer(
    max_epochs=20,
    accelerator='auto',
    devices=gpus,
    callbacks=[checkpoint_callback, early_stop_callback],
    log_every_n_steps=10,
    precision=16,
)

print(f"\nStarting training on {type(model).__name__}")
print(f"Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"Total params: {sum(p.numel() for p in model.parameters()):,}")

trainer.fit(model, data_module)

print(f"\nPhase 1 Complete!")
print(f"Best model: {checkpoint_callback.best_model_path}")
print(f"Best val_loss: {checkpoint_callback.best_model_score:.4f}")


print("\n" + "=" * 60)
print("PHASE 2: Full Fine-Tuning (Unfreezing Backbone)")
print("=" * 60)

print("\nUnfreezing backbone parameters...")
for param in model.backbone.parameters():
    param.requires_grad = True

model.learning_rate = 1e-5
print(f"Reduced learning rate to: {model.learning_rate}")

def new_configure_optimizers(self):
    return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)

import types
model.configure_optimizers = types.MethodType(new_configure_optimizers, model)

print(f"Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

checkpoint_callback_phase2 = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints/',
    filename='phase2-best-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min',
)

early_stop_callback_phase2 = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

trainer_phase2 = pl.Trainer(
    max_epochs=30,
    accelerator='auto',
    devices=gpus,
    callbacks=[checkpoint_callback_phase2, early_stop_callback_phase2],
    log_every_n_steps=10,
    precision=16,
)

print("\nContinuing training with unfrozen backbone...")
trainer_phase2.fit(model, data_module)

print(f"\nPhase 2 Complete!")
print(f"Best model: {checkpoint_callback_phase2.best_model_path}")
print(f"Best val_loss: {checkpoint_callback_phase2.best_model_score:.4f}")


print("\n" + "=" * 60)
print("TRAINING COMPLETE")
print("=" * 60)
print(f"""
Phase 1 (Head only):
  - Best checkpoint: {checkpoint_callback.best_model_path}
  - Best val_loss: {checkpoint_callback.best_model_score:.4f}

Phase 2 (Full fine-tuning):
  - Best checkpoint: {checkpoint_callback_phase2.best_model_path}
  - Best val_loss: {checkpoint_callback_phase2.best_model_score:.4f}

Next steps:
1. Test the model: trainer_phase2.test(model, data_module)
2. Make predictions on new images
3. Analyze errors and confusion matrix
""")

print("\n" + "=" * 60)
print("TESTING ON TEST SET")
print("=" * 60)

test_results = trainer_phase2.test(model, data_module)
print("\nTest Results:")
for key, value in test_results[0].items():
    print(f"  {key}: {value:.4f}")

torch.save({
    'model_state_dict': model.state_dict(),
    'model_type': type(model).__name__,
    'test_results': test_results[0],
}, 'final_deepfake_detector.pth')

print("\n✓ Final model saved: final_deepfake_detector.pth")

/usr/local/lib/python3.12/dist-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:751: Checkpoint directory /content/checkpoints exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


PHASE 1: Training Classification Head Only
Using 1 GPU(s).

Starting training on ResNetClassifier
Trainable params: 4,098
Total params: 23,512,130


/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/model_summary/model_summary.py:231: Precision 16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type             | Params | Mode 
--------------------------------------------------------
0 | loss_fn    | CrossEntropyLoss | 0      | train
1 | accuracy   | BinaryAccuracy   | 0      | train
2 | auroc      | BinaryAUROC      | 0      | train
3 | backbone   | ResNet           | 23.5 M | train
4 | classifier | Linear           | 4.1 K  | train
--------------------------------------------------------
4.1 K     Trainable params
23.5 M    Non-trainable params
23.5 M    Total params
94.049    Total estimated model params size (MB)
221       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


...


Phase 2 Complete!
Best model: /content/checkpoints/phase2-best-epoch=29-val_loss=0.17.ckpt
Best val_loss: 0.1708

TRAINING COMPLETE

Phase 1 (Head only):
  - Best checkpoint: /content/checkpoints/phase1-best-epoch=18-val_loss=0.49.ckpt
  - Best val_loss: 0.4897

Phase 2 (Full fine-tuning):
  - Best checkpoint: /content/checkpoints/phase2-best-epoch=29-val_loss=0.17.ckpt
  - Best val_loss: 0.1708

Next steps:
1. Test the model: trainer_phase2.test(model, data_module)
2. Make predictions on new images
3. Analyze errors and confusion matrix


TESTING ON TEST SET


Testing: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9233333468437195     │
│        test_auroc         │    0.02133333310484886    │
│         test_loss         │    0.17417973279953003    │
└───────────────────────────┴───────────────────────────┘


Test Results:
  test_loss: 0.1742
  test_acc: 0.9233
  test_auroc: 0.0213

✓ Final model saved: final_deepfake_detector.pth
